In [1]:
#[1] Import Modules

import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.cluster import OPTICS, cluster_optics_dbscan

import matplotlib
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap
from PIL import Image
from pylab import rcParams

df_saldo = pd.read_csv('ReplenishTrx.csv')
print ("Shape of the DataFrame: ", df_saldo.shape)

#Select Top 3 Data
df_saldo.head(3)


In [2]:
colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, 280))
print(colors)

[[0.         0.         0.5        1.        ]
 [0.         0.         0.5        1.        ]
 [0.         0.         0.51782531 1.        ]
 ...
 [0.51782531 0.         0.         1.        ]
 [0.5        0.         0.         1.        ]
 [0.5        0.         0.         1.        ]]


In [ ]:
#[2] Read CSV

df_saldo = pd.read_csv('ReplenishTrx.csv')
print ("Shape of the DataFrame: ", df_saldo.shape)
df_saldo.head(3) 

#Selecting the boundaries of the map from lattitude and longitude 
# L = lower
# u = Upper
llon=106.603905
ulon=107.094555
llat=-6.516244
ulat=-6.023535
 
print ("Shape of the DataFrame after selecting: ", df_saldo.shape)

In [ ]:
#Get Data
df_saldo_clus = df_saldo[[ "Longitude", "Latitude"]]
df_saldo_clus = Standardscaler().fit_transform(df_saldo_clus)

#Compute DBSCAN
_eps = 0.2
_min_samp = 10
db = DBSCAN(eps=_eps, min_samples= _min_samp, n_jobs=-l).fit(saldo_df_clus)

labels = db.labels_
saldo_df["ClusterDBSCAN"]=labels


#Export cluster to CSV
saldo_df.to_csv("DATA\\DBSCAN "+str(_eps)+" "+str(_min_samp)+".csv" index=False)

#Number of clusters in tabeLs, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len (set(labels))
n_noise_ = list(labels).count(-l)

#Print cLusters, noise, siLhoette score, timestamp
if n_clusters > 0:
    print(str(_eps) + ',' + str(_min_samp) + ',' + str(n_clusters_)
        + ',' +  str(n_noise_) + ','
        + str(metrics.silhouette_score(saldo_df_clus, labels, metric="euclidean",n_jobs=-l))
        + str(datetime.datetime.now()) )
else:
	print(str(_eps) + ',' + str(_min_samp) + ',' + str(n_clusters_)
	    + str(n_noise_)	+ ',' + str(datetime.datetime.now()) )  


In [ ]:
# Import File
saldo_df = pd.read_csv("DATA\\DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".csv")      
saldo_df = saldo_df.merge(pd.DataFrame({'CountReplenish':saldo_df.groupby(['MachineGroupCode', 'Longitude','Latitude','ClusterDBSCAN'])['MachineGroupCode'].count()}), left_on=['MachineGroupCode', 'Longitude','Latitude','ClusterDBSCAN'], right_index=True)
saldo_df = saldo_df.drop('Id', 1)
saldo_df = saldo_df.drop('DateReplenish', 1)
saldo_df = saldo_df.drop('MachineCode', 1)
saldo_df.drop_duplicates(subset=None, keep="first", inplace=True)


%matplotlib inline
rcParams['figure.figsize'] = (14,10)

#Create map
my_map = Basemap(llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
    urcrnrlon=ulon, urcrnrlat=ulat)
my_map.arcgisimage(service='World_Street_Map', xpixels = 3000)

labels = saldo_df["ClusterDBSCAN"].to_numpy()  
realClusterNum=len(set(labels)) - (1 if -1 in labels else 0) 
unique_labels = set(labels)

# Create color cluster
colors = plt.get_cmap('hsv')(np.linspace(0.0, 1.0, len(unique_labels)))

#Visualization
for clust_number in unique_labels: 
    c = (([0.4,0.4,0.4]) if clust_number == -1 else colors[clust_number])
    clust_set = saldo_df[saldo_df.ClusterDBSCAN == clust_number]  
    my_map.scatter(clust_set.Longitude, clust_set.Latitude, color =c,  marker='o', s= (clust_set.CountReplenish /100) +3 ,alpha = 0.65)
    
    if clust_number != -1:
        cenx=np.mean(clust_set.Longitude) 
        ceny=np.mean(clust_set.Latitude) 
        plt.text(cenx,ceny,str(clust_number), fontsize=3, color=c,
                path_effects=[path_effects.withStroke(linewidth=0.3, foreground="w")])

#Save Image
plt.title(r"ATM's in Jakarta Clustered : EPS="+str(_eps)+", MinSamples="+str(_min_samp), fontsize=14)        
plt.savefig("VISUALIZATION\\DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".png", dpi=400)
 

In [3]:
#[3] Clustering DBSCAN 

#Get Data
df_saldo_clus = df_saldo[[ "Longitude", "Latitude"]]
df_saldo_clus = StandardScaler().fit_transform(df_saldo_clus)

#Compute DBSCAN
_eps = 0.2
_min_samp = 10
db = OPTICS(min_samples=50, metric="euclidean").fit(df_saldo_clus)

#db = DBSCAN(eps=_eps, min_samples=_min_samp).fit(df_saldo_clus)
labels = db.labels_
df_saldo["ClusDb"]=labels
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

# Print Result
print('EPS: '+str( )+' & MinSamples: '+ str(_min_samp))
print('Estimated number of clusters: %d' % n_clusters)
print('Estimated number of noise points: %d' % n_noise)
"""
if n_clusters > 0:
    print("Silhouette Coefficient: %0.3f"
    % metrics.silhouette_score(df_saldo_clus, labels))
else:
   print("Silhouette Coefficient: -")
""" 

Shape of the DataFrame:  (153964, 6)
Shape of the DataFrame after selecting:  (153964, 6)


In [ ]:
#Get Data
df_saldo_clus = df_saldo[[ "Longitude", "Latitude"]]
df_saldo_clus = Standardscaler().fit_transform(df_saldo_clus)

#Compute DBSCAN
_eps = 0.2
_min_samp = 10
db = DBSCAN(eps=_eps, min_samples= _min_samp, n_jobs=-l).fit(saldo_df_clus)

labels = db.labels_
saldo_df["ClusterDBSCAN"]=labels


#Export cluster to CSV
saldo_df.to_csv("DATA\\DBSCAN "+str(_eps)+" "+str(_min_samp)+".csv" index=False)

#Number of clusters in tabeLs, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len (set(labels))
n_noise_ = list(labels).count(-l)

#Print cLusters, noise, siLhoette score, timestamp
if n_clusters > 0:
    print(str(_eps) + ',' + str(_min_samp) + ',' + str(n_clusters_)
        + ',' +  str(n_noise_) + ','
        + str(metrics.silhouette_score(saldo_df_clus, labels, metric="euclidean",n_jobs=-l))
        + str(datetime.datetime.now()) )
else:
	print(str(_eps) + ',' + str(_min_samp) + ',' + str(n_clusters_)
	    + str(n_noise_)	+ ',' + str(datetime.datetime.now()) )  


In [ ]:
# Import File
saldo_df = pd.read_csv("DATA\\DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".csv")      
saldo_df = saldo_df.merge(pd.DataFrame({'CountReplenish':saldo_df.groupby(['MachineGroupCode', 'Longitude','Latitude','ClusterDBSCAN'])['MachineGroupCode'].count()}), left_on=['MachineGroupCode', 'Longitude','Latitude','ClusterDBSCAN'], right_index=True)
saldo_df = saldo_df.drop('Id', 1)
saldo_df = saldo_df.drop('DateReplenish', 1)
saldo_df = saldo_df.drop('MachineCode', 1)
saldo_df.drop_duplicates(subset=None, keep="first", inplace=True)


%matplotlib inline
rcParams['figure.figsize'] = (14,10)

#Create map
my_map = Basemap(llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
    urcrnrlon=ulon, urcrnrlat=ulat)
my_map.arcgisimage(service='World_Street_Map', xpixels = 3000)

labels = saldo_df["ClusterDBSCAN"].to_numpy()  
realClusterNum=len(set(labels)) - (1 if -1 in labels else 0) 
unique_labels = set(labels)

# Create color cluster
colors = plt.get_cmap('hsv')(np.linspace(0.0, 1.0, len(unique_labels)))

#Visualization
for clust_number in unique_labels: 
    c = (([0.4,0.4,0.4]) if clust_number == -1 else colors[clust_number])
    clust_set = saldo_df[saldo_df.ClusterDBSCAN == clust_number]  
    my_map.scatter(clust_set.Longitude, clust_set.Latitude, color =c,  marker='o', s= (clust_set.CountReplenish /100) +3 ,alpha = 0.65)
    
    if clust_number != -1:
        cenx=np.mean(clust_set.Longitude) 
        ceny=np.mean(clust_set.Latitude) 
        plt.text(cenx,ceny,str(clust_number), fontsize=3, color=c,
                path_effects=[path_effects.withStroke(linewidth=0.3, foreground="w")])

#Save Image
plt.title(r"ATM's in Jakarta Clustered : EPS="+str(_eps)+", MinSamples="+str(_min_samp), fontsize=14)        
plt.savefig("VISUALIZATION\\DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".png", dpi=400)
 

In [4]:
#[3] Clustering DBSCAN 

#Get Data
df_saldo_clus = df_saldo[[ "Longitude", "Latitude"]]
df_saldo_clus = StandardScaler().fit_transform(df_saldo_clus)

#Compute DBSCAN
_eps = 0.2
_min_samp = 10
db = OPTICS(min_samples=50, metric="euclidean").fit(df_saldo_clus)

#db = DBSCAN(eps=_eps, min_samples=_min_samp).fit(df_saldo_clus)
labels = db.labels_
df_saldo["ClusDb"]=labels
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

# Print Result
print('EPS: '+str( )+' & MinSamples: '+ str(_min_samp))
print('Estimated number of clusters: %d' % n_clusters)
print('Estimated number of noise points: %d' % n_noise)
"""
if n_clusters > 0:
    print("Silhouette Coefficient: %0.3f"
    % metrics.silhouette_score(df_saldo_clus, labels))
else:
   print("Silhouette Coefficient: -")
""" 

KeyboardInterrupt: 

In [ ]:
#[4] Visualization 
%matplotlib inline
rcParams['figure.figsize'] = (14,10)

# Create map
my_map = Basemap(llcrnrlon=llon, llcrnrlat=llat,urcrnrlon=ulon, urcrnrlat=ulat)
my_map.arcgisimage(service='World_Street_Map', xpixels = 2000)

# Create color cluster
colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, n_clusters + 1))


# Looping Cluster
for clust_number in set(labels):
    c=(([0.4,0.4,0.4]) if clust_number == -1 else colors[np.int(clust_number)])
    clust_set = df_saldo[df_saldo.ClusDb == clust_number]                    
    my_map.scatter(clust_set.Longitude, clust_set.Latitude, color =c,  marker='o', s= 10, alpha = 0.65)
    if clust_number != -1:
        cenx=np.mean(clust_set.Longitude) 
        ceny=np.mean(clust_set.Latitude) 
        plt.text(cenx,ceny,str(clust_number), fontsize=30, color='red',)
        print ("Cluster "+str(clust_number)+', Average Mean Debit Amount: '+ str(np.mean(clust_set.DebitAmount)))
        print ("Cluster "+str(clust_number)+', Average Mean Count Replenish: '+ str(np.mean(clust_set.CountReplenish)))
plt.title(r"ATM di Jakarta Clustered: $ \epsilon="+str(_eps)+"$ MinSamples="+str(_min_samp), fontsize=14)        

# Save Image
plt.savefig("DBSCAN_VISUAL.png", dpi=300)

In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def ProsesDBSCAN(_eps, _min_samp):
     # Import Library DBSCAN
    from sklearn.cluster import DBSCAN
    import sklearn.utils
    from sklearn.preprocessing import StandardScaler
    
    import matplotlib.patheffects as path_effects
    saldo_df = pd.read_csv("DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".csv") 

    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    from pylab import rcParams
    %matplotlib inline
    rcParams['figure.figsize'] = (14,10)
    my_map = Basemap(llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
        urcrnrlon=ulon, urcrnrlat=ulat)
    my_map.arcgisimage(service='World_Street_Map', xpixels = 3000)

    
    labels = saldo_df["ClusterDBSCAN"].to_numpy()  
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum = len(set(labels)) 

    # To create a color map
    colors = plt.get_cmap('hsv')(np.linspace(0.0, 1.0, clusterNum))
         
    #Visualization1
    for clust_number in set(labels):
        c=(([0.4,0.4,0.4]) if clust_number == -1 else colors[np.int(clust_number)])
        #c1 = (([0.4,0.4,0.4]) if clust_number == -1 else get_cmap(clust_number))
        clust_set = saldo_df[saldo_df.ClusterDBSCAN == clust_number]                    
        my_map.scatter(clust_set.Longitude, clust_set.Latitude, color =c,  marker='o', s= 5, alpha = 0.65)
        if clust_number != -1:
            cenx=np.mean(clust_set.Longitude) 
            ceny=np.mean(clust_set.Latitude) 
            plt.text(cenx,ceny,str(clust_number), fontsize=3, color=c,
                    path_effects=[path_effects.withStroke(linewidth=0.5, foreground="w")])
            #print ("Cluster "+str(clust_number)+', Average Mean Debit Amount: '+ str(np.mean(clust_set.DebitAmount)))
            #print ("Cluster "+str(clust_number)+', Average Mean Count Replenish: '+ str(np.mean(clust_set.CountReplenish)))
    plt.title(r"ATM's in Jakarta Clustered : EPS="+str(_eps)+" MinSamples="+str(_min_samp), fontsize=14)        
    plt.savefig("DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".png", dpi=400)
    
ProsesDBSCAN(0.01,50)



In [ ]:
colors = plt.get_cmap('hsv')(np.linspace(0.0, 1.0, 1100))
df = pd.DataFrame(data=colors)
print(colors)
print(df)

In [ ]:
def rand_cmap(nlabels, type='soft', first_color_black=True, last_color_black=False, verbose=True):
    """
    Creates a random colormap to be used together with matplotlib. Useful for segmentation tasks
    :param nlabels: Number of labels (size of colormap)
    :param type: 'bright' for strong colors, 'soft' for pastel colors
    :param first_color_black: Option to use first color as black, True or False
    :param last_color_black: Option to use last color as black, True or False
    :param verbose: Prints the number of labels and shows the colormap. True or False
    :return: colormap for matplotlib
    """
    from matplotlib.colors import LinearSegmentedColormap
    import colorsys
    import numpy as np


    if type not in ('bright', 'soft'):
        print ('Please choose "bright" or "soft" for type')
        return

    if verbose:
        print('Number of labels: ' + str(nlabels))

    # Generate color map for bright colors, based on hsv
    if type == 'bright':
        randHSVcolors = [(np.random.uniform(low=0.0, high=1),
                          np.random.uniform(low=0.2, high=1),
                          np.random.uniform(low=0.9, high=1)) for i in range(nlabels)]

        # Convert HSV list to RGB
        randRGBcolors = []
        for HSVcolor in randHSVcolors:
            randRGBcolors.append(colorsys.hsv_to_rgb(HSVcolor[0], HSVcolor[1], HSVcolor[2]))

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]

        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Generate soft pastel colors, by limiting the RGB spectrum
    if type == 'soft':
        low = 0.6
        high = 0.95
        randRGBcolors = [(np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high)) for i in range(nlabels)]

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]
        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Display colorbar
    if verbose:
        from matplotlib import colors, colorbar
        from matplotlib import pyplot as plt
        fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))

        bounds = np.linspace(0, nlabels, nlabels + 1)
        norm = colors.BoundaryNorm(bounds, nlabels)

        cb = colorbar.ColorbarBase(ax, cmap=random_colormap, norm=norm, spacing='proportional', ticks=None,
                                   boundaries=bounds, format='%1i', orientation=u'horizontal')

    return random_colormap


In [ ]:
def VisualisasiDBSCAN(_eps, _min_samp):
     # Import Library DBSCAN
    import numpy as np
    import pandas as pd
        
    from sklearn.cluster import DBSCAN
    import sklearn.utils
    from sklearn.preprocessing import StandardScaler
    
    import matplotlib.patheffects as path_effects
    saldo_df = pd.read_csv("DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".csv") 

    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    from pylab import rcParams
    %matplotlib inline
    rcParams['figure.figsize'] = (14,10)
    my_map = Basemap(llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
        urcrnrlon=ulon, urcrnrlat=ulat)
    my_map.arcgisimage(service='World_Street_Map', xpixels = 3000)

    
    labels = saldo_df["ClusterDBSCAN"].to_numpy()  
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum = len(set(labels)) 
    colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 'olive', 'goldenrod', 'lightcyan', 'navy']
    vectorizer = np.vectorize(lambda x: colors[x % len(colors)])
    # To create a color map
    colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, clusterNum))
    
    
    coloor = rand_cmap(clusterNum, type='soft', first_color_black=True, last_color_black=False, verbose=True)
    #Visualization1
    for clust_number in set(labels):
        #c=(([0.4,0.4,0.4]) if clust_number == -1 else colors[np.int(clust_number)])
        c = (([0.4,0.4,0.4]) if clust_number == -1 else coloor(clust_number)[:3])
        clust_set = saldo_df[saldo_df.ClusterDBSCAN == clust_number]                    
        my_map.scatter(clust_set.Longitude, clust_set.Latitude, color=c,  marker='o', s= 5, alpha = 0.65)
        if clust_number != -1:
            cenx=np.mean(clust_set.Longitude) 
            ceny=np.mean(clust_set.Latitude) 
            #plt.text(cenx,ceny,str(clust_number), fontsize=3, color=c,
            #        path_effects=[path_effects.withStroke(linewidth=0.5, foreground="w")])
    plt.title(r"ATM's in Jakarta Clustered : EPS="+str(_eps)+" MinSamples="+str(_min_samp), fontsize=14)        
    plt.savefig("VISUALIZATION\\DBSCAN_"+str(_eps)+"_"+str(_min_samp)+".png", dpi=400)
     
VisualisasiDBSCAN(0.01,50)